In [3]:
pwd

'c:\\Users\\USER\\AppData\\Local\\Programs\\Microsoft VS Code'

In [1]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")
[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

c:\Users\USER\anaconda3\envs\key_bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [12]:
import torch
from transformers import BertModel


# KoBERT 모델과 토크나이저 불러오기
model = BertModel.from_pretrained('skt/kobert-base-v1',output_attentions=True)

# 문서 예제
text = "국가와 기업은 밀접한 상호작용을 통해 서로에게 큰 영향을 미치고 있습니다. 기업이 성공적으로 성장하면 국가는 경제적 안정과 발전을 경험할 수 있습니다. 따라서 국가 경제를 발전시키기 위해서는 기업의 창립과 지속이 매우 중요한 과제인데요. 통계청의 ‘2022년 기업생멸행정통계’​를 통해  우리나라 기업의 생존율을 알아보도록 하겠습니다!"
# 토큰화 및 토큰 ID로 변환
input_ids = tokenizer.encode(text, add_special_tokens=True)
input_ids_tensor = torch.tensor([input_ids])

# 모델을 사용하여 출력 계산
with torch.no_grad():
    outputs = model(input_ids_tensor)

# Attention 값 추출
attention = outputs.attentions  # 각 레이어와 헤드별 Attention 값이 담긴 리스트

# 마지막 레이어의 Attention 값 평균 계산
last_layer_attention = attention[-1]  # 마지막 레이어의 Attention
mean_attention = last_layer_attention.mean(dim=1)  # 헤드에 대한 평균

# 토큰별로 평균 Attention Score 계산
token_attention_scores = mean_attention.squeeze().mean(dim=0)

# Attention Score가 가장 높은 토큰 추출
top_tokens = token_attention_scores.topk(5)  # 상위 5개 토큰

# 추출된 키워드 출력
keywords = [tokenizer.decode([input_ids[idx]]) for idx in top_tokens.indices]
print(keywords)



['[SEP]', '생존', '안정', '멸', '발전']


In [3]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# KoBERT 모델과 토크나이저 불러오기
model = BertModel.from_pretrained('skt/kobert-base-v1')

# 문서 예제
text = "여기에 분석하고자 하는 문서를 넣으세요. 예를 들어, 이 문장은 BERT 모델을 사용하여 키워드를 추출하는 예제입니다."

# 토큰화 및 토큰 ID로 변환
input_ids = tokenizer.encode(text, add_special_tokens=True)
input_ids_tensor = torch.tensor([input_ids])

# 모델을 사용하여 출력 계산
with torch.no_grad():
    outputs = model(input_ids_tensor)

# 어텐션 스코어 추출
attention = outputs.attentions
last_layer_attention = attention[-1]
mean_attention = last_layer_attention.mean(dim=1)
token_attention_scores = mean_attention.squeeze().mean(dim=0).numpy()

# TF-IDF 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform([text])
tfidf_scores = np.array(tfidf.sum(axis=0)).flatten()

# 단어 사전 구축
feature_names = tfidf_vectorizer.get_feature_names_out()

# 어텐션 스코어와 TF-IDF 스코어 결합
combined_scores = token_attention_scores * tfidf_scores

# 결합된 스코어를 기반으로 상위 키워드 추출
num_keywords = 5
top_indices = combined_scores.argsort()[-num_keywords:]
keywords = [feature_names[i] for i in top_indices]

print("Extracted Keywords:", keywords)


TypeError: 'NoneType' object is not subscriptable